In [1]:
import pandas as pd
import matplotlib.pyplot as plt  
import numpy as np 
import seaborn as sns 

In [2]:
data = pd.read_csv('SBAnational.csv')

/var/folders/83/skc59h713n1f5fpwznbhnkr80000gn/T/ipykernel_13812/1533426435.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('SBAnational.csv')


##### On crée un nouveau dataframe où les colonnes qui sont inutiles pour les prédictions sont supprimées ainsi que les variables suspectées de faire du dataleaking.

In [3]:
data_lite = data.drop(['DisbursementDate', 'BalanceGross', 'LoanNr_ChkDgt', 'ChgOffDate', 'DisbursementGross', 'ChgOffPrinGr'], axis=1)
data_lite.head(2)

,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,...,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,MIS_Status,GrAppv,SBA_Appv
0,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,84,...,2.0,0,0,1,0,N,Y,P I F,"$60,000.00","$48,000.00"
1,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,60,...,2.0,0,0,1,0,N,Y,P I F,"$40,000.00","$32,000.00"


#####  Vérifier les types des variables, les valeurs manquantes ou suspectes.
On vérifie les valeurs uniques pour chaque variable du dataframe ainsi que les 'missing values'.

In [4]:
print(data_lite.isnull().sum())

data_lite.dtypes


Name               14
City               30
State              14
Zip                 0
Bank             1559
BankState        1566
NAICS               0
ApprovalDate        0
ApprovalFY          0
Term                0
NoEmp               0
NewExist          136
CreateJob           0
RetainedJob         0
FranchiseCode       0
UrbanRural          0
RevLineCr        4528
LowDoc           2582
MIS_Status       1997
GrAppv              0
SBA_Appv            0
dtype: int64


Name              object
City              object
State             object
Zip                int64
Bank              object
BankState         object
NAICS              int64
ApprovalDate      object
ApprovalFY        object
Term               int64
NoEmp              int64
NewExist         float64
CreateJob          int64
RetainedJob        int64
FranchiseCode      int64
UrbanRural         int64
RevLineCr         object
LowDoc            object
MIS_Status        object
GrAppv            object
SBA_Appv          object
dtype: object

On transforme les dates en type 'DateTime'

In [5]:
data_lite['ApprovalDate'] = pd.to_datetime(data_lite['ApprovalDate'],format='mixed', dayfirst= False) #pour convertir des données en date

pd.options.display.date_dayfirst = True
data_lite['ApprovalDate'] = data_lite['ApprovalDate'].dt.strftime('%d/%m/%Y')  #pour formater les dates déjà converties dans un format specifique
data_lite

#retirer A de 1976A dans les cases concernées
data_lite['ApprovalFY'] = data_lite['ApprovalFY'].str.replace('A', '')
data_lite['ApprovalFY'].dtypes #object

dtype('O')

Vérifier les valeurs uniques des colonnes. Si les valeurs différent des modalités officielles, on les remplace par NaN.

In [6]:
data_lite['MIS_Status'].unique() 

# on supprime les valeurs manquantes de la target
data_lite.dropna(subset=['MIS_Status'], inplace=True)

# encodage de la target
data_lite['MIS_Status'] = data_lite['MIS_Status'].map({'P I F': 0, 'CHGOFF': 1})
data_lite.MIS_Status.dtype



dtype('int64')

In [7]:
data_lite['RevLineCr'].unique() # renvoie:array(['N', '0', 'Y', 'T',......, 'Q']

valeurs_odd = [ 'NaN', 'nan','0', 'T','`', ',', '1', 'C', '3', '2', 'R', '7','A', '5', '.', '4', '-', 'Q']

data_lite['RevLineCr'] = data_lite['RevLineCr'].replace(valeurs_odd, np.nan)

data_lite['RevLineCr'].unique()

array(['N', nan, 'Y'], dtype=object)

In [ ]:
data_lite['Term'].unique() #valeurs numériques 

In [9]:
data_lite['NewExist'].unique()

#relever les lignes avec la valeur 0 de ma colonne
rows_with_zero = data_lite.loc[data_lite['NewExist']==0]
rows_with_zero

#remplacer les 0 par des Nan
data_lite['NewExist'] = data_lite['NewExist'].replace(0, np.nan)
data_lite['NewExist'].unique()

array([ 2.,  1., nan])

In [10]:
data_lite['FranchiseCode'].unique()

array([    1,     0, 15100, ...,  2899, 18701, 15930])

In [11]:
data_lite['UrbanRural'].unique() #remplace 0 par NaN

#remplacer les 0 par des Nan
data_lite['UrbanRural'] = data_lite['UrbanRural'].replace(0, np.nan)
data_lite['UrbanRural'].unique()


array([nan,  1.,  2.])

In [11]:
data_lite['LowDoc'].unique()
valeurs_odd = ['C', '1','S', 'R', 'A', '0']
data_lite['LowDoc'] = data_lite['RevLineCr'].replace(valeurs_odd, np.nan)
data_lite['LowDoc'].unique()

array(['N', nan, 'Y'], dtype=object)

In [12]:
data_lite.dtypes

Name              object
City              object
State             object
Zip                int64
Bank              object
BankState         object
NAICS              int64
ApprovalDate      object
ApprovalFY        object
Term               int64
NoEmp              int64
NewExist         float64
CreateJob          int64
RetainedJob        int64
FranchiseCode      int64
UrbanRural         int64
RevLineCr         object
LowDoc            object
MIS_Status         int64
GrAppv            object
SBA_Appv          object
dtype: object

In [13]:
#enlever le symbole $ de sbvappv

data_lite["SBA_Appv"] = data_lite["SBA_Appv"].str.replace("$", "").str.replace(",", "").str.strip()
data_lite["SBA_Appv"] = data_lite["SBA_Appv"].astype(float)
data_lite.head(2)
data_lite.dtypes

Name              object
City              object
State             object
Zip                int64
Bank              object
BankState         object
NAICS              int64
ApprovalDate      object
ApprovalFY        object
Term               int64
NoEmp              int64
NewExist         float64
CreateJob          int64
RetainedJob        int64
FranchiseCode      int64
UrbanRural         int64
RevLineCr         object
LowDoc            object
MIS_Status         int64
GrAppv            object
SBA_Appv         float64
dtype: object

In [14]:
#la franchise code assigne un code lorsque l'entreprise est une franchise, ou 1 ou 0 lorsque ce n'est pas une franchise
# on peut remplacer les valeurs par '0': pas de franchise et '1': franchise en 2 étapes

data_lite.loc[data_lite['FranchiseCode']  <=1, 'FranchiseCode'] = 0

data_lite.loc[data_lite['FranchiseCode'] > 0, 'FranchiseCode'] = 1

data_lite.tail(2)

,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,...,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,MIS_Status,GrAppv,SBA_Appv
899162,"MARUTAMA HAWAII, INC.",HONOLULU,HI,96830,BANK OF HAWAII,HI,0,27/02/1997,NaN,60,...,1.0,0,0,0,0,N,N,1,"$75,000.00",60000.0
899163,PACIFIC TRADEWINDS FAN & LIGHT,KAILUA,HI,96734,CENTRAL PACIFIC BANK,HI,0,27/02/1997,NaN,48,...,2.0,0,0,0,0,N,N,0,"$30,000.00",24000.0


In [15]:
#supprimer les ',' dans la colonne SBA_Appv
data_lite['SBA_Appv'] = data_lite['SBA_Appv'].str.replace(',', '').str.strip()
data_lite['SBA_Appv'] = data_lite['SBA_Appv'].astype(float)
data_lite['SBA_Appv'].dtypes

AttributeError: Can only use .str accessor with string values!

#### Enregistre le dataframe data_lite pour le réutiliser dans le notebook modelisation


In [36]:
data_lite.to_csv('data.csv', index=False)

